In [6]:
println(sc.version)

In [7]:
val wreg = """\w+""".r

In [8]:
val sentences = sc.textFile("file:///home/jovyan/work/data/corpus/*/*.txt", minPartitions=128).map {
  case line =>
    wreg.findAllIn(line).filter { _.forall { _.isLetter } }.map { word => word.toLowerCase }.toVector
}.filter{ _.size > 10 }.persist()

In [16]:
println(sentences.count())
println(sentences.take(20).mkString("\n") )

4036418
Vector(please, take, a, look, at, the, important, information, in, this, header)
Vector(we, encourage, you, to, keep, this, file, on, your, own, disk, keeping, an)
Vector(electronic, path, open, for, the, next, readers, do, not, remove, this)
Vector(this, etext, was, done, by, a, number, of, anonymous, volunteers, of, the)
Vector(gutenberg, projec, to, whom, we, owe, a, great, deal, of, thanks, and, to)
Vector(if, our, etexts, manage, to, make, it, to, an, average, about, million)
Vector(then, we, will, have, given, about, billion, etexts, away, when, the)
Vector(we, are, now, trying, to, release, all, our, books, one, month, in, advance)
Vector(the, official, release, date, of, all, project, gutenberg, etexts, is, at)
Vector(midnight, central, time, of, the, last, day, of, the, stated, month, a)
Vector(and, editing, by, those, who, wish, to, do, so, to, be, sure, you, have, an)
Vector(in, the, first, week, of, the, next, month, since, our, ftp, program, has)
Vector(a, bug, in,

In [10]:
val Array(train, test) = sentences.randomSplit(Array(0.25, 0.75), seed=333L)

In [11]:
train.persist()
test.persist()

MapPartitionsRDD[6] at randomSplit at <console>:18

In [12]:
import org.apache.spark.mllib._
import org.apache.spark.mllib.feature._

In [13]:
val model = new Word2Vec().
    setLearningRate(0.1).
    setMinCount(10).
    setNumPartitions(16).
    setVectorSize(100).
    fit(sentences)

In [20]:
val mother = model.transform("king").toArray
val woman = model.transform("man").toArray
val man = model.transform("woman").toArray

In [34]:
val queenLike = mother.zip(woman).
    map{ case (a: Double, b: Double) => a - b }.zip(man).
    map { case (a, b) => a  + b}

In [39]:
val queen = model.transform("queen").toArray

In [40]:
def norm(vec: Array[Double]): Double = {
    math.sqrt(vec.map { x => x * x }.sum)
}

In [41]:
queen.zip(queenLike).map { case (a, b) => a * b }.sum / norm(queen) / norm(queenLike)

0.3718098920732813